<a href="https://colab.research.google.com/github/hgabrali/CIFAR-10-Image-Classification-A-Transfer-Learning-Approach-with-ResNet50/blob/main/notebooks/CIFAR10_ResNet50_TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIFAR-10 Image Classification: A Transfer Learning Approach with ResNet50

## 📌 Project Overview
Transitioning from the grayscale simplicity of Fashion MNIST, this project tackles the **CIFAR-10** dataset—a benchmark in computer vision consisting of 60,000 $32 \times 32$ color images across 10 mutually exclusive classes.

The primary objective was to leverage **Transfer Learning** using the **ResNet50** architecture, demonstrating the efficiency of using pre-trained weights (ImageNet) to solve complex image classification tasks even with relatively low-resolution inputs.